In [ ]:
try:
    # mount google drive
    from google.colab import drive  # nopep8
    drive_path = "/content/drive"
    drive.mount(drive_path)
    drive_folder = drive_path + "/MyDrive/dtm/"
    using_colab = True
except ModuleNotFoundError:
    # Assume we are not on google colab,
    drive_folder = "data/"
    using_colab = False
    pass

# Params and Init

In [ ]:
!pip -qq install --upgrade pip
!pip -qq install plyfile deepdiff talos GitPython numba
!pip -qq install --upgrade imgaug
import talos
from deepdiff import DeepDiff
import re
import numpy as np
from progressbar import progressbar, ProgressBar
import sys
import os
import cv2
import math
import json
import importlib
import matplotlib.pyplot as plt
import tensorflow as tf
import git
import keras
import keras.backend as K
import datetime
import imgaug.augmenters as iaa
cuda_paths=!ls /usr/local | grep cuda-
os.environ['CUDA_HOME'] = os.path.join("/usr/local/",cuda_paths[-1])

In [ ]:
if using_colab:
    !git -C /content/deep_ga/ pull || git clone https://github.com/InigoMoreno/deep_ga
    sys.path.append('/content/deep_ga')
    import deep_ga  # nopep8
    deep_ga=importlib.reload(deep_ga)
else:
    MODULE_PATH = "deep_ga/deep_ga/__init__.py"
    MODULE_NAME = "deep_ga"
    git.cmd.Git(MODULE_NAME).pull()
    spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
    deep_ga = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = deep_ga
    spec.loader.exec_module(deep_ga)


In [ ]:
p={
  "resolution" : 1,                      # resolution of map [meters per pixel]
  "mapLength"  : 80,                    # size of one side of the map [meters]
  "minSlopeThreshold"  : 0.5,            # minimum slope to be counted [proportion]
  "maxNanPercentage"   : 5/100,           # maximum percentage of NaNs in a patch [%]
  "minSlopePercentage" : 10/100,          # minimum percentage of slope in a patch [%]
  "maxSlopePercentage" : 40/100,          # maximum percentage of slope in a patch [%]
  "stdPatchShift"          : 15,          # standard deviation of shift between to patches [m]
   "raycastHeight": 2,
   "booleanDist": False
}
p["mapLengthPixels"]=math.ceil(p["mapLength"]/p["resolution"])
deep_ga.set_scale(p["stdPatchShift"])
p["augment_a"] = iaa.Sequential([
    iaa.PerspectiveTransform(scale=0.01),
    iaa.BlendAlphaSimplexNoise(iaa.Add(0.2), upscale_method="cubic", size_px_max=4),
])

hyperparams = {
    # "input": ["raw", "fixsobel", "PConv", "SymConv", "Conv"],
    "input": ["SymConv"],
    # "batch_size": [32, 64, 128, 256],
    "batch_size": [32],
    "mobileNet_alpha": [0.35, 0.5, 1.0],
    # "mobileNet_alpha": [1.0],
    # "mobileNet_pooling": ["avg", "max", None],
    "mobileNet_pooling": ["avg"],
    # "mobileNet_weights": ["imagenet"],
    "mobileNet_weights": [None],
    # "firstLayerSize": [4096, 1280, 1000],"
    "firstLayerSize": [0],
    # "dropout": [0, 0.2, 0.5],
    "dropout": [0],
    # "secondLayerSize": [1000, 500, 100],
    "secondLayerSize": [500,100,15,0],
    # "activation": ["relu", None],
    "activation": ["relu"],
    # "sharedWeights": [True, False],
    "sharedWeights": [True],
    # "loss": ["MSE", "MAE", "DOOMSE", "PCL", "BCE"],
    "loss": ["DOOMSE"],
    "optimizer": ["Adam"],
    # "learning_rate": [0.01, 0.001, 0.0001],
    "learning_rate": [0.0001],
    # "learnEnding": [True]
    "learnEnding": [False]
}

keys= [k for k,v in hyperparams.items() if len(v)>1]
experiment_path = os.path.relpath(os.path.join(drive_folder,"talos_oxia_"+"__".join(keys)))

first_hyperparams={k:v[0] for k,v in hyperparams.items()}

# Dataset preparation

In [ ]:
!pip -q install gdal wget
import gdal, wget, os

!pip -q install gdal wget
import gdal, wget, os

filename="oxia_planum.IMG"

filename=os.path.join(drive_folder, "oxia_planum.IMG")

#get file if it does not exist
#from https://www.uahirise.org/dtm/dtm.php?ID=ESP_037070_1985
url = "https://www.uahirise.org/PDS/DTM/ESP/ORB_037000_037099/ESP_037070_1985_ESP_037136_1985/DTEEC_037070_1985_037136_1985_L01.IMG"
if not os.path.isfile(filename):
  wget.download(url,out = filename)

#transform to numpy array
gdata=gdal.Open(filename)
height=gdata.GetRasterBand(1).ReadAsArray()
height[height<-1e38]=np.nan
height = cv2.resize(height, None, interpolation = cv2.INTER_AREA,
                    fx=gdata.GetGeoTransform()[1] / p["resolution"],
                    fy=-gdata.GetGeoTransform()[5] / p["resolution"])


filename=os.path.join(drive_folder, "jezero.IMG")

#get file if it does not exist
#from https://www.uahirise.org/dtm/dtm.php?ID=ESP_023247_1985
url = "https://www.uahirise.org/PDS/DTM/ESP/ORB_023200_023299/ESP_023247_1985_ESP_022957_1985/DTEEC_023247_1985_022957_1985_U01.IMG"
if not os.path.isfile(filename):
  wget.download(url,out = filename)

#transform to numpy array
gdata=gdal.Open(filename)
height2=gdata.GetRasterBand(1).ReadAsArray()
height2[height2<-1e38]=np.nan
height2 = cv2.resize(height2, None, interpolation = cv2.INTER_AREA,
                    fx=gdata.GetGeoTransform()[1] / p["resolution"],
                    fy=-gdata.GetGeoTransform()[5] / p["resolution"])


In [ ]:
plt.clf()
height-=np.nanmin(height)
plt.imshow(height,cmap="viridis")
max=np.nanmax(height)
plt.xlabel("x [m]")
plt.ylabel("y [m]")
cb=plt.colorbar(ticks=[0,max])
cb.set_label("elevation [m]",labelpad=-10, rotation=270)
# plt.savefig("oxia.pdf", dpi=500, bbox_inches = 'tight',
#     pad_inches = 0)
plt.show();

In [ ]:

def get_batch(batch_size, dem, p, seed=None):
    """Get a batch of patches for training

    Args:
        batch_size (int): Size of the batch
        dem (numpy array): elevation map
        p (dict): parameters
        seed (int, optional): Seed to use for randomness. Defaults to None.

    Returns:
        (tuple): tuple containing
            patches_a (numpy array): left patches of the batch
            patches_b (numpy array): right patches of the batch
            distances (double): distances between left and right patches
    """
    # initialize random
    random = np.random.RandomState(seed)

    # initialize data
    patches_a = np.empty(
        (batch_size, p["mapLengthPixels"], p["mapLengthPixels"]))
    patches_b = np.empty(
        (batch_size, p["mapLengthPixels"], p["mapLengthPixels"]))
    distances = np.empty((batch_size,))

    for i in range(batch_size):
        # find first patch
        patch_a = None
        while patch_a is None:
            xa = random.uniform(0, dem.shape[0])
            ya = random.uniform(0, dem.shape[1])
            patch_a = deep_ga.get_patch(dem, xa, ya, p)
        if "augment_a" in p.keys() and p["augment_a"] is not None:
            patch_a = p["augment_a"].augment_image(patch_a)
        if "raycastHeight" in p.keys() and p["raycastHeight"] is not None:
            patch_a = deep_ga.raycast_occlusion(patch_a, p["raycastHeight"])
        patches_a[i, :, :] = patch_a - np.nanmean(patch_a)

        # find second patch close to first
        patch_b = None
        while patch_b is None:
            if p["booleanDist"]:
                if random.choice([True, False]):
                    dx, dy = 0, 0
                else:
                    angle = random.uniform(0, 2 * math.pi)
                    dx = p["stdPatchShift"] * math.cos(angle)
                    dy = p["stdPatchShift"] * math.sin(angle)
            else:
                dx, dy = random.normal(
                    scale=p["stdPatchShift"] / p["resolution"], size=2)
            patch_b = deep_ga.get_patch(dem, xa + dx, ya + dy, p)
        if "augment_b" in p.keys() and p["augment_b"] is not None:
            patch_b = p["augment_b"].augment_image(patch_b)
        patches_b[i, :, :] = patch_b - np.nanmean(patch_b)

        # compute output function
        distances[i] = np.linalg.norm([dx, dy]) * p["resolution"]

    return (patches_a, patches_b, distances)



In [ ]:
# p["stdPatchShift"]=0
# seed=140
# seed=160
# seed=580
seed=np.random.randint(1000)
print(seed)
patches_a, patches_b,distances = get_batch(1, height, p, seed=seed)


for i in range(1):
  print(i)
  W=p["mapLength"]/2
  # print(distances[i])
  plt.subplot(1,2,1)
  plt.title("Original DTM patch")
  
  plt.xlabel("x [m]")
  plt.ylabel("y [m]")
  plt.imshow(patches_b[i,:,:],extent=(-W,W,-W,W))


  W=p["mapLength"]/2
  plt.subplot(1,2,2)
  plt.title("Artificial occlusion result")
  plt.xlabel("x [m]")
  # plt.ylabel("y [m]")
  plt.imshow(patches_a[i,:,:],extent=(-W,W,-W,W))
  # plt.savefig(f"oxia_occlusion.pdf",  dpi=500, bbox_inches = 'tight',
  #   pad_inches = 0)
  plt.show()
# plt.hist(distances)
# plt.show()

# Model Training

In [ ]:
if using_colab: 
    %load_ext tensorboard
    %tensorboard --logdir {os.path.join(experiment_path,"logs")}

In [ ]:
import datetime

def talos_model(x_train, y_train, x_val, y_val, params):
  global experiment_path, keys, model

  values=[str(params[k]) for k in keys]
  i="__".join(values)
  i=i.replace(".","_")
  print(i)

  # get data generator
  tgenerator=deep_ga.PatchDataGenerator(10000,height,batch_size=params["batch_size"],params=p)

  # get validation data generator
  vp=p.copy()
  vp["augment_a"]=None
  vgenerator=deep_ga.PatchDataGenerator(10000,height2,batch_size=params["batch_size"],params=vp)

  with open(os.path.join(experiment_path, f"{i}_params.json"), 'w') as fp:
      json.dump(params, fp)

  callbacks=[]

  # stop early if loss does not improve
  callbacks.append(keras.callbacks.EarlyStopping(
      monitor='val_loss', patience=25, verbose=1, restore_best_weights=True,min_delta=0.1))
  
  # assert that weights are finite
  callbacks.append(deep_ga.AssertWeightsFinite())

  # plot prediction
  callbacks.append(deep_ga.PlotPrediction(
      folder=os.path.join(experiment_path, f"{i}_plots"),
      tgen=tgenerator,
      vgen=vgenerator,
      save=not using_colab
    ))
  
    
  # tensorboard callback
  logdir = os.path.join(experiment_path,"logs", f"{i}")
  callbacks.append(tf.keras.callbacks.TensorBoard(logdir))

  # get model
  input_a = keras.Input(shape=(
      p["mapLengthPixels"], p["mapLengthPixels"]), name="patch_a")
  input_b = keras.Input(
      shape=(p["mapLengthPixels"], p["mapLengthPixels"]), name="patch_b")

  model = deep_ga.get_model(params,input_a,input_b)
  model = deep_ga.compile_model(model, y_val, params)

  out = model.fit(tgenerator, 
                  validation_data=vgenerator,
                  epochs=100, 
                  workers=6,
                  callbacks=callbacks,
                  verbose=(1 if using_colab else 2)
                  )
                  
  x,y=tgenerator.get_batch(320)
  yp=model.predict(x)[:,0]
  print(f"self mse loss = {keras.losses.MSE(y,yp)}")

  with open(os.path.join(experiment_path, f"{i}_structure.json"), 'w') as fp:
      fp.write(model.to_json())
  model.save_weights(os.path.join(experiment_path, f"{i}_weights.h5"))
  model.save(os.path.join(experiment_path, f"{i}.h5"))
  model.save(os.path.join(experiment_path, f"{i}"),include_optimizer=False)

  return out, model


In [ ]:

fake_generator=deep_ga.PatchDataGenerator(10,height,batch_size=320,params=p)
(fake_a, fake_b), fake_out = fake_generator[0]

i=0


import shutil
if os.path.exists(experiment_path):
  shutil.rmtree(experiment_path)

scan_object = talos.Scan(
    x=[fake_a, fake_b], y=fake_out,
    params=hyperparams,
    model=talos_model,
    experiment_name=experiment_path,
    time_limit=(datetime.datetime.now() +
                datetime.timedelta(hours=40)).strftime("%Y-%m-%d %H:%M"),
    # reduction_method='correlation',
    # reduction_interval=10,
    # reduction_window=20,
    # reduction_threshold=0.2,
    # reduction_metric='NMSE',
    minimize_loss=True,
    save_weights=True,
    print_params=True,
    random_method='quantum'
)


In [ ]:

patches_a, patches_b,distances = get_batch(1, height, p)
sb=model.get_layer("single_branch")
# sb.predict([patches_a])
for layer in sb.layers:
  print(layer.name)

sbf=keras.models.Model(inputs=sb.inputs, outputs=sb.layers[2].output)
Xf=sbf.predict([patches_a])
X=sb.predict([patches_a])
print(X.shape)
# plt.imshow(Xf[0,:,:,:]*10)
plt.scatter(range(len(X[0,:])),X[0,:]);
plt.show()
# model.predict([patches_a, patches_b])


In [ ]:
import pickle


def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, protocol=2)


def project_object(obj, *attributes):
    out = {}
    for a in attributes:
        out[a] = getattr(obj, a)
    return out


pickle_path = os.path.join(
    experiment_path, datetime.datetime.now().strftime("%Y%m%d_%H%M.pickle"))
t = project_object(scan_object, 'params', 'saved_models',
                   'saved_weights', 'data', 'details', 'round_history')
save_object(t, pickle_path)



Copy this on the console to prevent termination:
```
function ClickConnect() {
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect, 60000)
```

Interesting resources: 
 - Hyperparameter optimization with talos
 - Tinyfying Neural Network with uTensor or TinyML

